In [38]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report, confusion_matrix, recall_score, roc_auc_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
import string
import nltk

from nltk.corpus import stopwords as sw
from nltk.corpus import wordnet as wn
from nltk import wordpunct_tokenize
from nltk import WordNetLemmatizer
from nltk import sent_tokenize
from nltk import pos_tag

## Pipeline

In [2]:
class MyPreprocessor(BaseEstimator, TransformerMixin):

    def __init__(self,
                 lower=True, strip=True):
        self.lower      = lower
        self.strip      = strip
        self.stopwords  = set(sw.words('english'))
        self.punct      = set(string.punctuation)
        self.lemmatizer = WordNetLemmatizer()

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [
            list(self.tokenize(doc)) for doc in X
        ]

    def tokenize(self, document):
        for sent in sent_tokenize(document):
            for token, tag in pos_tag(wordpunct_tokenize(sent)):
                token = token.lower() if self.lower else token
                token = token.strip() if self.strip else token
                token = token.strip('_') if self.strip else token
                token = token.strip('*') if self.strip else token

                if token in self.stopwords:
                    continue
                
                if token in ["thanks", "gold", "edit", "obligatory", "gild", "gilded"]:
                    continue

                if all(char in self.punct for char in token):
                    continue

                lemma = self.lemmatize(token, tag)
                yield lemma

    def lemmatize(self, token, tag):
        tag = {
            'N': wn.NOUN,
            'V': wn.VERB,
            'R': wn.ADV,
            'J': wn.ADJ
        }.get(tag[0], wn.NOUN)

        return self.lemmatizer.lemmatize(token, tag)

In [18]:
def identity(words):
    return words

In [3]:
def identity(words):
    return words

model = Pipeline([
    ('normalizer', MyPreprocessor()),
    ('vectorizer', TfidfVectorizer(tokenizer=identity, preprocessor=None, lowercase=False)),
    ('bayes', MultinomialNB()),
])

## Fit & Score

In [ ]:
df = pd.read_csv('data/df_gildtest.csv')
df.loc[df['gilded'] >= 1, 'target'] = 1
df.loc[df['gilded'] == 0, 'target'] = 0
df['target'] = df['target'].astype(int)
X = df['body'].tolist()
y = df['target'].tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [7]:
model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('normalizer', MyPreprocessor(lower=True, strip=True)),
                ('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=False, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function identity at 0x1a22a7e598>,
                                 use_idf=True, vocabulary=None)),
                ('bayes',
                 MultinomialNB(alpha

In [8]:
model.score(X_test, y_test)

0.9444444444444444

In [14]:
df = pd.read_csv('data/askreddit_12.csv')
df.loc[df['gilded'] >= 1, 'target'] = 1
df.loc[df['gilded'] == 0, 'target'] = 0
df['target'] = df['target'].astype(int)
X = df['body'].tolist()
y = df['target'].tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [16]:
model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('normalizer', MyPreprocessor(lower=True, strip=True)),
                ('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=False, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function identity at 0x1a30c077b8>,
                                 use_idf=True, vocabulary=None)),
                ('bayes',
                 MultinomialNB(alpha

In [11]:
model.score(X_test, y_test)

0.9993679502139245

In [20]:
y_pred = model.predict(X_test)

In [23]:
from sklearn.metrics import classification_report
|
print(classification_report(y_test, y_pred, digits=6))

              precision    recall  f1-score   support

           0   0.999368  1.000000  0.999684     20555
           1   0.000000  0.000000  0.000000        13

    accuracy                       0.999368     20568
   macro avg   0.499684  0.500000  0.499842     20568
weighted avg   0.998736  0.999368  0.999052     20568



In [24]:
from sklearn.metrics import recall_score

print(recall_score(y_test, y_pred))

0.0


In [32]:
df = pd.read_csv('data/askreddit_tops.csv')
df.loc[df['gilded'] >= 1, 'target'] = 1
df.loc[df['gilded'] == 0, 'target'] = 0
df['target'] = df['target'].astype(int)
X = df['body'].tolist()
y = df['target'].tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

In [26]:
model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('normalizer', MyPreprocessor(lower=True, strip=True)),
                ('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=False, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function identity at 0x1a1a9b37b8>,
                                 use_idf=True, vocabulary=None)),
                ('bayes',
                 MultinomialNB(alpha

In [6]:
model.score(X_test, y_test)

0.96

In [7]:
y_pred = model.predict(X_test)

In [10]:
print(classification_report(y_test, y_pred, digits=6))

              precision    recall  f1-score   support

           0   0.960000  1.000000  0.979592       240
           1   0.000000  0.000000  0.000000        10

    accuracy                       0.960000       250
   macro avg   0.480000  0.500000  0.489796       250
weighted avg   0.921600  0.960000  0.940408       250



//anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
print(recall_score(y_test, y_pred))

0.0


## GridSearch (hyperparameter tuning)

In [45]:
search = GridSearchCV(model, param_grid={
    'vectorizer__max_features': [None, 20000],
    'bayes__alpha': [0.0, 1.0],
})

In [47]:
search.fit(X, y)

//anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:485: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
//anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:485: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
//anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:485: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
//anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:485: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' 

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('normalizer',
                                        MyPreprocessor(lower=True, strip=True)),
                                       ('vectorizer',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=False,
                                                        max_df=1.0,
                                                        max_features=None,
                                     

In [48]:
search.cv_results_

{'mean_fit_time': array([51.6429824 , 50.86932818, 49.44741535, 47.64937782]),
 'std_fit_time': array([12.57975814, 11.59230337, 11.37953344, 11.113098  ]),
 'mean_score_time': array([24.63709569, 25.46759383, 24.00808144, 23.86866434]),
 'std_score_time': array([10.97257155, 11.43602056, 10.62008113, 10.68915501]),
 'param_bayes__alpha': masked_array(data=[0.0, 0.0, 1.0, 1.0],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_vectorizer__max_features': masked_array(data=[None, 20000, None, 20000],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'bayes__alpha': 0.0, 'vectorizer__max_features': None},
  {'bayes__alpha': 0.0, 'vectorizer__max_features': 20000},
  {'bayes__alpha': 1.0, 'vectorizer__max_features': None},
  {'bayes__alpha': 1.0, 'vectorizer__max_features': 20000}],
 'split0_test_score': array([0.99934366, 0.99938013, 0.99938013, 0.99938013]),
 'split1

## Cost-Sensitive Learning

In [27]:
# Predict class probabilities
prob_y_2 = model.predict_proba(X_test)
 
# Keep only the positive class
prob_y_2 = [p[1] for p in prob_y_2]

In [28]:
prob_y_2[:5]

[0.00873199495949682,
 0.010704635281685125,
 0.0025225025184457297,
 0.0024610625057940303,
 0.0076587196922102]

In [29]:
roc_auc_score(y_test, prob_y_2)

0.39208333333333334

In [20]:
model_bal = Pipeline([
    ('normalizer', MyPreprocessor()),
    ('vectorizer', TfidfVectorizer(tokenizer=identity, preprocessor=None, lowercase=False)),
    ('bayes', RandomForestClassifier(random_state=0, n_jobs=-1, class_weight="balanced")),
])

In [21]:
model_bal.fit(X_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('normalizer', MyPreprocessor(lower=True, strip=True)),
                ('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=False, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words...
                 RandomForestClassifier(bootstrap=True, class_weight='balanced',
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
        

In [22]:
pred_bal = model_bal.predict(X_test)

In [25]:
# Is our model still predicting just one class?
print( np.unique( pred_bal ) )
 
# How's our accuracy?
print( accuracy_score(y_test, pred_bal) )
 
# What about AUROC?
prob_y_bal = model_bal.predict_proba(X_test)
prob_y_bal = [p[1] for p in prob_y_bal]
print( roc_auc_score(y_test, prob_y_bal) )

[0]
0.96
0.430625


In [30]:
## Weighted Classes

w = 50 # The weight for the positive class

model_wei = Pipeline([
    ('normalizer', MyPreprocessor()),
    ('vectorizer', TfidfVectorizer(tokenizer=identity, preprocessor=None, lowercase=False)),
    ('forest', RandomForestClassifier(random_state=0, n_jobs=-1, class_weight={0: 1, 1: w})),
])


In [34]:
model_wei.fit(X_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('normalizer', MyPreprocessor(lower=True, strip=True)),
                ('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=False, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words...
                 RandomForestClassifier(bootstrap=True,
                                        class_weight={0: 1, 1: 50},
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                

In [35]:
pred_wei = model_wei.predict(X_test)

# Is our model still predicting just one class?
print( np.unique( pred_wei ) )
 
# How's our accuracy?
print( accuracy_score(y_test, pred_wei) )
 
# What about AUROC?
prob_y_wei = model_wei.predict_proba(X_test)
prob_y_wei = [p[1] for p in prob_y_wei]
print( roc_auc_score(y_test, prob_y_wei) )

[0 1]
0.86
0.33756510416666663


In [36]:
print(classification_report(y_test, pred_wei, digits=6))

              precision    recall  f1-score   support

           0   0.960674  0.890625  0.924324       192
           1   0.045455  0.125000  0.066667         8

    accuracy                       0.860000       200
   macro avg   0.503064  0.507812  0.495495       200
weighted avg   0.924065  0.860000  0.890018       200



In [39]:
print(confusion_matrix(y_test, pred_wei))

[[171  21]
 [  7   1]]


In [41]:
tn, fp, fn, tp = confusion_matrix(y_test, pred_wei).ravel()
print(tp, fp, tn, fn)

1 21 171 7
